In [2]:
# Import packages
import os
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.manifold import TSNE

import mindscope_utilities
import mindscope_utilities.visual_behavior_ophys as ophys

from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

pd.set_option('display.max_columns', 500)

# Mount Google Drive
try:
  import google.colab
  google.colab.drive.mount('/content/drive')
  cache_directory = "/temp/"  # Note: this path must exist on your local drive
  save_data_directory = '/content/drive/MyDrive/'
  IN_COLAB = True
except:
  cache_directory = "./cache/"  # Note: this path must exist on your local drive
  save_data_directory = './'
  IN_COLAB = False

In [ ]:
# Get preprocessed dataset
# fname = "allen_visual_behavior_2p_change_detection_familiar_novel_image_sets.parquet"
# url = "https://ndownloader.figshare.com/files/28470255"
# if not os.path.isfile(fname):
#   try:
#     r = requests.get(url)
#   except requests.ConnectionError:
#     print("!!! Failed to download data !!!")
#   else:
#     if r.status_code != requests.codes.ok:
#       print("!!! Failed to download data !!!")
#     else:
#       with open(fname, "wb") as fid:
#         fid.write(r.content)

# filename = "allen_visual_behavior_2p_change_detection_familiar_novel_image_sets.parquet"
# preprocessed_data = pd.read_parquet(filename)
# preprocessed_data.head(5)

# print(preprocessed_data.session_type.unique())
# print(np.sort(preprocessed_data.query('cre_line == "Vip-IRES-Cre" and session_type == "OPHYS_3_images_B"')['mouse_id'].unique()))
# print(np.sort(preprocessed_data.query('cre_line == "Vip-IRES-Cre" and session_type == "OPHYS_4_images_A"')['mouse_id'].unique()))

In [4]:
# Get high-level csv files from S3 storage
cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=cache_directory)

ophys_session_table.csv: 100%|██████████| 227k/227k [00:00<00:00, 1.23MMB/s] 
behavior_session_table.csv: 100%|██████████| 1.21M/1.21M [00:00<00:00, 5.23MMB/s]
ophys_experiment_table.csv: 100%|██████████| 610k/610k [00:00<00:00, 3.12MMB/s] 
ophys_cells_table.csv: 100%|██████████| 4.29M/4.29M [00:00<00:00, 12.6MMB/s]


In [5]:
# Load session table (and experiment table if desired) as data frames
session_table = cache.get_ophys_session_table()
#ophys_session_ids = session_table.index
#experiment_table = cache.get_ophys_experiment_table()
print(session_table.session_type.unique())
print(session_table.cre_line.unique())

['OPHYS_1_images_A' 'OPHYS_2_images_A_passive' 'OPHYS_3_images_A'
 'OPHYS_4_images_B' 'OPHYS_5_images_B_passive' 'OPHYS_6_images_B'
 'OPHYS_1_images_B' 'OPHYS_2_images_B_passive' 'OPHYS_3_images_B'
 'OPHYS_4_images_A' 'OPHYS_6_images_A' 'OPHYS_5_images_A_passive'
 'OPHYS_1_images_G' 'OPHYS_3_images_G' 'OPHYS_2_images_G_passive'
 'OPHYS_4_images_H' 'OPHYS_5_images_H_passive' 'OPHYS_6_images_H']
['Sst-IRES-Cre' 'Vip-IRES-Cre' 'Slc17a7-IRES2-Cre']


In [11]:
# Select a subset of mouse cre lines and sessions
target_cre_lines = ["Sst-IRES-Cre"]
target_session_types = ["OPHYS_1_images_B", "OPHYS_3_images_B", "OPHYS_4_images_A", "OPHYS_6_images_A"];
session_table_subset = session_table.query('cre_line in @target_cre_lines and session_type in @target_session_types')

# Determine which mice in this subset have data from ALL sessions
#session_table_subset.groupby(['mouse_id','session_type'])['age_in_days'].mean()
mouse_sessions = pd.DataFrame(session_table_subset.groupby(['mouse_id',])['session_type'].unique())
mouse_sessions['has_all_sessions'] = mouse_sessions.apply(lambda x: len(x['session_type'])==4, axis=1)
target_mouse_ids = mouse_sessions.query('has_all_sessions').index
print(target_mouse_ids)

Int64Index([470784, 476970, 477202, 480753, 485688, 489056, 498972], dtype='int64', name='mouse_id')


In [30]:
np.float16(0.0000001)

1e-07

In [31]:
# Loop over sessions for this mouse
# note that there may be different subsets of cells in different sessions
# some sessions will have multiple experiments (imaging planes), some may only have one
#all_data = []
#for mouse_id in target_mouse_ids[:3]:
for mouse_id in target_mouse_ids:
  print('--------------')
  ophys_session_ids = session_table.query('mouse_id == @mouse_id').index
  print(f'Mouse ID {mouse_id} includes {len(ophys_session_ids)} sessions')
  mouse_data = []
  for ophys_session_id in ophys_session_ids:
    # Get session metadata
    session_metadata = session_table.loc[ophys_session_id]
    # Download experiments for each session
    experiments = {} # clear experiments dictionary
    ophys_experiment_ids = session_metadata['ophys_experiment_id']
    print(f'Session ID {ophys_session_id} includes {len(ophys_experiment_ids)} experiments: {np.array(ophys_experiment_ids)}')
    for ophys_experiment_id in ophys_experiment_ids:
      experiments[ophys_experiment_id] = cache.get_behavior_ophys_experiment(ophys_experiment_id)
    # stimulus table is shared for all experiments in a session
    stimulus_table = experiments[ophys_experiment_ids[0]].stimulus_presentations.drop(columns = ['image_set'])  # 'image_set' is unnecessary
    # Download neural data for each experiment
    neural_data = []
    for ophys_experiment_id in experiments.keys():
      this_experiment = experiments[ophys_experiment_id]
      this_experiment_neural_data = ophys.build_tidy_cell_df(this_experiment)
      # add some columns with metadata for the experiment
      metadata_keys = [
          'ophys_experiment_id',
          'ophys_session_id',
          'targeted_structure',
          'imaging_depth',
          'equipment_name',
          'cre_line',
          'mouse_id',
          'sex',
      ]
      for metadata_key in metadata_keys:
        this_experiment_neural_data[metadata_key] = this_experiment.metadata[metadata_key]

      # append the data for this experiment to a list
      neural_data.append(this_experiment_neural_data)
      print(f'Experiment {ophys_experiment_id} includes {len(this_experiment_neural_data["cell_specimen_id"].unique())} cells.')
      #: {np.array(this_experiment_neural_data["cell_specimen_id"].unique())}')

    # concatate the list of dataframes into a single dataframe
    neural_data = pd.concat(neural_data)

    full_etr_l = []
    # Get the experiment-long timeseries for each cell
    for cell_specimen_id in tqdm(neural_data['cell_specimen_id'].unique()):
      # calculate the event triggered response for this cell to every stimulus
      full_etr_this_cell = mindscope_utilities.event_triggered_response(
          data=neural_data.query('cell_specimen_id == @cell_specimen_id'),
          t='timestamps',
          y='dff', #'dff', 'events', 'filtered_events'
          event_times=stimulus_table['start_time'],
          t_before=0.5,
          t_after=0.75,
          output_sampling_rate=30, # Hz (so 30 = every 33 ms)
      )
      # add a column identifying the cell_specimen_id
      full_etr_this_cell['cell_specimen_id'] = cell_specimen_id
      # append to our list
      full_etr_l.append(full_etr_this_cell)

    # concatenate our list of dataframes into a single dataframe
    full_etr = pd.concat(full_etr_l)

    # cast these numeric columns to int and float, respectively
    full_etr['event_number'] = full_etr['event_number'].astype(int)
    full_etr['event_time'] = full_etr['event_number'].astype(float)

    # Compute the average firing rate
    # Default is the average from 0 to 500 ms
    # We will also compute a baseline-corrected version 'dff_bc'
    average_0_to_500 = full_etr.query('time >= 0 and time <= .5').groupby(['cell_specimen_id', 'stimulus_presentations_id'])[['dff']].mean().reset_index()
    average_n133_to_n33 = full_etr.query('time > -.15 and time < 0').groupby(['cell_specimen_id', 'stimulus_presentations_id'])[['dff']].mean().reset_index()
    dff_bc = average_0_to_500['dff']-average_n133_to_n33['dff']

    average_responses = average_0_to_500.merge(
      stimulus_table,
      on='stimulus_presentations_id',
      how='left'
    )
    average_responses['dff_stim500'] = average_0_to_500['dff']
    average_responses['dff_bc'] = dff_bc
    average_responses['trace'] = full_etr.groupby(['cell_specimen_id','stimulus_presentations_id'])['dff'].apply(list).values
    average_responses['trace_timestamps'] = full_etr.groupby(['cell_specimen_id','stimulus_presentations_id'])['time'].apply(list).values
    additional_metadata = neural_data.groupby('cell_specimen_id')[metadata_keys].max().reset_index()
    average_responses = average_responses.merge(additional_metadata,on='cell_specimen_id',how='left')
    mouse_data.append(average_responses)

  # Finished a mouse!
  mouse_data = pd.concat(mouse_data)

  # Add some additional metadata
  lol = session_table.query('mouse_id == @mouse_id')
  lol = lol.groupby(['ophys_session_id']).max().reset_index()
  lol['exposure_level'] = lol['session_number']>3
  lol = lol.replace({'exposure_level': {False:'familiar',True:'novel'}})
  mouse_data = mouse_data.merge(
      lol[['mouse_id','ophys_session_id','session_number','exposure_level']],
      on=['mouse_id','ophys_session_id'],
      how='left'
  )

  # remove some unnecessary data
  unneeded = ['start_time','stop_time','duration','image_index','start_frame','end_frame','equipment_name','trace_timestamps']
  mouse_data = mouse_data.drop(unneeded,axis=1)

  # change data types for memory efficiency
  change_type_dict = {'cell_specimen_id': 'category',
                      'stimulus_presentations_id':'int16',
                      'dff': 'float16',
                      'dff_stim500': 'float16',
                      'image_name': 'category',
                      'dff_bc': 'float16',
                      'ophys_experiment_id': 'category',
                      'ophys_session_id': 'category',
                      'targeted_structure': 'category',
                      'imaging_depth': 'int16',
                      'cre_line': 'category',
                      'mouse_id': 'category',
                      'sex': 'category',
                      'session_number': 'int8',
                      'exposure_level': 'category'
                      }
  mouse_data = mouse_data.astype(change_type_dict)

  mouse_data.to_pickle(f'{save_data_directory}{target_cre_lines[0][:3]}_mouse{mouse_id}.pkl')
  #all_data.append(mouse_data)

# Finished all mice!
#all_data = pd.concat(all_data)

--------------
Mouse ID 470784 includes 6 sessions
Session ID 954981981 includes 1 experiments: [955276580]
Experiment 955276580 includes 17 cells.


100%|██████████| 17/17 [00:10<00:00,  1.64it/s]


Session ID 955991376 includes 1 experiments: [956903375]
Experiment 956903375 includes 19 cells.


100%|██████████| 19/19 [00:15<00:00,  1.26it/s]


Session ID 957189583 includes 1 experiments: [957652800]
Experiment 957652800 includes 18 cells.


...

100%|██████████| 7/7 [00:05<00:00,  1.30it/s]


Session ID 1013000229 includes 1 experiments: [1013063357]


behavior_ophys_experiment_1013063357.nwb: 100%|██████████| 265M/265M [00:11<00:00, 23.2MMB/s]    


Experiment 1013063357 includes 10 cells.


100%|██████████| 10/10 [00:08<00:00,  1.23it/s]


In [ ]:
# To access categorical information
#all_data1['image_name'].cat.categories
#all_data1['image_name'].cat.codes

# The trace timestamps
#trace_timestamps = np.linspace(-0.5,0.7333333333333334,38)